In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import analysis.yinf as yinf
import analysis.products as products
import analysis.util as util
from data.util import *
import os, sys, re
from scipy.stats import gaussian_kde, linregress
ML_DIR = os.path.expandvars("$SCRATCH/mlreco_cell/")
NETS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(ML_DIR))])))
SIM_DIR = os.path.expandvars("$SCRATCH/larsim/")
SIMS = dict(enumerate(sorted([d for d in os.listdir(os.path.expandvars(SIM_DIR)) if "reco" in d])))

def parse_stats_dir(stats_dir = './stats'):
    stats_files = []
    for root, dirs, files in os.walk(stats_dir):
        rel_path = os.path.relpath(root, stats_dir)
        if rel_path.count('/') != 1: continue
        slash_i = rel_path.find('/')
        header = [rel_path[:slash_i], rel_path[slash_i+1:]]
        for f in files:
            epoch, _ = inf_file_info(f)
            thres = re.findall('thres([0-9]*[.]?[0-9]+)', f)
            if len(thres) == 1: 
                thres = float(thres[0])
            else:
                thres = -1
            stats_files.append(header+[epoch, thres, root+'/'+f])
    return pd.DataFrame(stats_files, columns=['net', 'inf', 'epoch', 'thres', 'file'])

STATS = parse_stats_dir().sort_values(by=['epoch'])

def select_stats(net=None, inf=None, thres=None, first=0, last=-1, stride=1, stats_df=STATS):
    if net is not None:
        stats_df = stats_df[stats_df.net==net]
    if inf is not None:
        stats_df = stats_df[stats_df.inf==inf]
    if thres is not None:
        stats_df = stats_df[stats_df.thres==thres]
    if last == -1:
        epochs = sorted(set(stats_df.epoch))[first::stride]
    else:
        epochs = sorted(set(stats_df.epoch))[first:last+1:stride]
    return list(stats_df[stats_df.epoch.isin(epochs)]['file'])


In [ ]:
NETS

In [ ]:
SIMS

In [ ]:
with np.load("stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-BeamCosmic-tpc1_stats-thres0.17-comptrue-N500-all.npz",
             allow_pickle=True) as f:
    stats = f["event_stats"]
Index, Auc = [], []
for evt in stats:
    Index.append(evt[:2])
    Auc.append(util.SP_score((evt[2], evt[3])))
Index = np.array(Index)
Auc = np.array(Auc)
args = np.argsort(Auc)[:10]
print(Index[args])
print(Auc[args])
fig, ax = plt.subplots()
ax.hist(Auc)
ax.set_xlabel("AUC")
ax.set_ylabel("N Events")
ax.set_title("Electron Inference AUC Distribution")
plt.show()
    

In [ ]:
with np.load("stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-BeamCosmic-tpc1_stats-thres0.17-comptrue-N500-all.npz",
             allow_pickle=True) as f:
    stats = f["event_stats"]
inf_thres = 18
Eff_inf, Pur_inf = [], []
Eff_sp, Pur_sp = [], []
for evt in stats[:]:
    index, event = evt[:2]
    try:
        with np.load(SIM_DIR+"reco_1GeV_BeamCosmic_parsed/reco_BeamCosmic_0%d_spacepoint-TPC12.npz"%index) as f:
            voxel_spacepoint = {tuple(pt): None for pt in f["arr_%d"%event]}
    except Exception:
        continue
    with np.load(SIM_DIR+"reco_1GeV_BeamCosmic_parsed/reco_BeamCosmic_0%d_depo-TPC12.npz"%index) as f:
        voxel_truth = {tuple(pt[:3]): pt[3]*1 for pt in f["arr_%d"%event]}
    voxel_truth, voxel_spacepoint = util.filter_voxels_coord(((-100, 700), None, None), *util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True))
    voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
    pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
    Pur_sp.append(pur_sp)
    Eff_sp.append(eff_sp)
    Pur_inf.append(evt[2][inf_thres])
    Eff_inf.append(evt[3][inf_thres])

fig, axes = plt.subplots(2)
axes[0].scatter(Eff_inf, Eff_sp, s=2)
axes[1].scatter(Pur_inf, Pur_sp, s=2)
for ax in axes:
    ax.set_xlim((0,1))
    ax.set_ylim((0,1))
    ax.set_xlabel("Network (thres=0.36)")
    ax.set_ylabel("Spacepoint")
axes[0].set_title("Sensitivity")
axes[1].set_title("Purity")
fig.text(0.2, 0.8, "Network mean: %.3f"%np.mean(Eff_inf))
fig.text(0.2, 0.7, "Spacepoint mean: %.3f"%np.mean(Eff_sp))
fig.text(0.2, 0.3, "Network mean: %.3f"%np.mean(Pur_inf))
fig.text(0.2, 0.2, "Spacepoint mean: %.3f"%np.mean(Pur_sp))
fig.suptitle("Spacepoint vs. Network \n file: epoch24-BeamCosmic-tpc1_stats-thres0.17-comptrue-N500-v2.npz")
plt.show()

In [ ]:
with np.load("stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-Electron_stats-thres0.50-comptrue-N500-v2.npz",
             allow_pickle=True) as f:
    stats = f["event_stats"]
inf_thres = 18
Eff_inf, Pur_inf = [], []
Eff_sp, Pur_sp = [], []
for evt in stats[:]:
    index, event = evt[:2]
    try:
        with np.load(SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_%d_spacepoint-TPC12.npz"%index) as f:
            voxel_spacepoint = {tuple(pt): None for pt in f["arr_%d"%event]}
    except Exception:
        continue
    with np.load(SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_%d_energy-TPC12.npz"%index) as f:
        voxel_truth = {tuple(pt[:3]): pt[3]*1/3 for pt in f["arr_%d"%event]}
    voxel_truth, voxel_spacepoint = util.filter_voxels_coord(((-100, 700), None, None), *util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True))
    voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
    pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
    Pur_sp.append(pur_sp)
    Eff_sp.append(eff_sp)
    Pur_inf.append(evt[2][inf_thres])
    Eff_inf.append(evt[3][inf_thres])

fig, axes = plt.subplots(2)
axes[0].scatter(Eff_inf, Eff_sp, s=2)
axes[1].scatter(Pur_inf, Pur_sp, s=2)
for ax in axes:
    ax.set_xlim((0,1))
    ax.set_ylim((0,1))
    ax.set_xlabel("Network (thres=0.36)")
    ax.set_ylabel("Spacepoint")
axes[0].set_title("Sensitivity")
axes[1].set_title("Purity")
fig.text(0.2, 0.8, "Network mean: %.3f"%np.mean(Eff_inf))
fig.text(0.2, 0.7, "Spacepoint mean: %.3f"%np.mean(Eff_sp))
fig.text(0.2, 0.3, "Network mean: %.3f"%np.mean(Pur_inf))
fig.text(0.2, 0.2, "Spacepoint mean: %.3f"%np.mean(Pur_sp))
fig.suptitle("Spacepoint vs. Network \n file: epoch24-Electron_stats-thres0.50-comptrue-N500-v2.npz")
plt.show()

In [ ]:
index = 91
event = 1
lims = ((100, 600), (550, 700), (120, 150))
yinf.plot_yinf_simple(plt.figure(), None, event, 0.35, 1, 0.5/3, lims, 
                      xy_dir=SIM_DIR+"reco_1GeV_BeamCosmic_xy/", yinf_file=ML_DIR+"singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-batch0-reco_BeamCosmic_0%d_yinf-TPC1.npy"%index)
products.plot_spacepoint(plt.figure(), None ,event , (8, 4, 4), 1, 0.5/3, ((lims[0][0]-62.5, lims[0][1]-62.5), lims[1], lims[2]), 
                    spacepoint_file=SIM_DIR+"reco_1GeV_BeamCosmic_parsed/reco_BeamCosmic_0%d_spacepoint-TPC12.npz"%index, truth_file=SIM_DIR+"reco_1GeV_BeamCosmic_parsed/reco_BeamCosmic_0%d_depo-TPC12.npz"%index)

In [ ]:

yinf.plot_yinf_simple(plt.figure(), None, 7, 0.35, E_scale=1/3, true_thres=0.5/3,
                      xy_dir=SIM_DIR+"reco_1GeV_ElectronWire_xy-v3_3/", yinf_file=ML_DIR+"singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-batch0-reco_singleElectron_953_yinf-TPC1.npy")
products.plot_spacepoint(plt.figure(), None, 7, (8, 4, 4), E_scale=1/3, true_thres=0.5/3, plot_lims=((400, 600), None, None),
                    spacepoint_file=SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_953_spacepoint-TPC12.npz", truth_file=SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_953_energy-TPC12.npz")

In [ ]:
yinf.plot_yinf_simple(plt.figure(), None, 5, 0.35, E_scale=1/3, true_thres=0.1,
                      xy_dir=SIM_DIR+"reco_1GeV_ElectronWire-downsample4-2-2_xy-v3", yinf_file=ML_DIR+"singleElectronWire-downsample(4,2,2)v3_ghost3D-165-LabelWeighted10-p0.9/inference/epoch40-batch0-reco_singleElectron_781_yinf-TPC1.npy")
products.plot_spacepoint(plt.figure(), None, 5, (4, 2, 2), E_scale=1/3, true_thres=0.1, plot_lims=((-100, 1400), None, None),
                    spacepoint_file=SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_781_spacepoint-TPC12.npz", truth_file=SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_781_energy-TPC12.npz")

Following cells are for tuning geom

In [ ]:
from geom.pdsp import get_APA_geom, VOXEL_SIZE, DOWNSAMPLE, get_TPC_inverse_coords_transform, get_TPC_box
from geom.util import voxelize
parsed_dir=SIM_DIR+"reco_1GeV_BeamCosmic_parsed"
product="depo"
def join_tpcs(index, event, t0s=(-6, -6), tpcs=[1, 2]):
    R0 = get_TPC_box(tpcs[0])[0]
    tpcs_key = int("".join(map(str, tpcs)))
    fd = filter_fd(files_info(os.listdir(parsed_dir), (1, 2)), lambda k,v: k[1]==product and k[0]==index)
    fd_keys = fd.keys()
    #print(fd)
    for index in fd_keys:
        tpc_fd = files_info(fd[index], (0,), tpc_info)
        Pts = []
        for tpc in tpcs:
            tpc = (tpc, )
            if tpc not in tpc_fd: break
            Pts.append([])
            with np.load(parsed_dir+'/'+tpc_fd[tpc][0], allow_pickle=True) as parsed_f:
                i = 0
                while "arr_%d"%i in parsed_f:
                    Pts[-1].append(parsed_f["arr_%d"%i])
                    i += 1
        if len(Pts) != len(tpcs): continue
        out_f = tpc_fd[(tpcs[0],)][0].replace("TPC%d"%tpcs[0], "TPC%d"%tpcs_key)
        Transforms = [get_TPC_inverse_coords_transform(tpc, VOXEL_SIZE, R0, t0)[0] for tpc, t0 in zip(tpcs, t0s)]
        data = []
        for tpc in range(len(tpcs)):
            data.extend(Transforms[tpc](Pts[tpc][event]))
        return data

files = [f for f in os.listdir(parsed_dir) if file_info(f)[2] == "spacepoint"][:10]
print(files)
T0s, Eff_sp, Pur_sp = [], [], []
for ti in np.linspace (-7, -5, 5):
    for tj in np.linspace(-7, -5, 5):
        Eff_sp_tmp, Pur_sp_tmp = [], []
        for sp_f in files:
            index = file_info(sp_f)[1]
            for event in range(5):
                with np.load(parsed_dir+'/'+sp_f) as f:
                    voxel_spacepoint = {tuple(pt): None for pt in f["arr_%d"%event]}
                joined = join_tpcs(index, event, (ti, tj))
                voxel_truth = {tuple(pt[:3]): pt[3]*1/3 for pt in joined}
                #voxel_truth, voxel_spacepoint = util.filter_voxels_coord(((-100, 700), None, None), *util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True))
                #voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
                pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
                Pur_sp_tmp.append(pur_sp)
                Eff_sp_tmp.append(eff_sp)
        T0s.append((ti, tj))
        Eff_sp.append(np.mean(Eff_sp_tmp))
        Pur_sp.append(np.mean(Pur_sp_tmp))


In [ ]:
print(T0s)
print(Eff_sp)
print(Pur_sp)
fig, axes = plt.subplots(2)
axes[0].plot(Eff_sp, 'bo')
axes[1].plot(Pur_sp, 'bo')

In [ ]:
Eff_sp = np.array(Eff_sp)
Pur_sp = np.array(Pur_sp)
T0s=np.array(T0s)
args = np.argsort(Eff_sp)[:10:-1]
print(T0s[args])
print(Eff_sp[args])
print(Pur_sp[args])

In [ ]:
from geom.pdsp import get_APA_geom, VOXEL_SIZE, DOWNSAMPLE, get_TPC_inverse_coords_transform, get_TPC_box
from geom.util import voxelize
parsed_dir=SIM_DIR+"reco_1GeV_Electron_parsed-v3"
product="energy"
def join_tpcs(index, event, t0s=(-6, -6), tpcs=[1, 2]):
    R0 = get_TPC_box(tpcs[0])[0]
    tpcs_key = int("".join(map(str, tpcs)))
    fd = filter_fd(files_info(os.listdir(parsed_dir), (1, 2)), lambda k,v: k[1]==product and k[0]==index)
    fd_keys = fd.keys()
    #print(fd)
    for index in fd_keys:
        tpc_fd = files_info(fd[index], (0,), tpc_info)
        Pts = []
        for tpc in tpcs:
            tpc = (tpc, )
            if tpc not in tpc_fd: break
            Pts.append([])
            with np.load(parsed_dir+'/'+tpc_fd[tpc][0], allow_pickle=True) as parsed_f:
                i = 0
                while "arr_%d"%i in parsed_f:
                    Pts[-1].append(parsed_f["arr_%d"%i])
                    i += 1
        if len(Pts) != len(tpcs): continue
        out_f = tpc_fd[(tpcs[0],)][0].replace("TPC%d"%tpcs[0], "TPC%d"%tpcs_key)
        Transforms = [get_TPC_inverse_coords_transform(tpc, VOXEL_SIZE, R0, t0)[0] for tpc, t0 in zip(tpcs, t0s)]
        data = []
        for tpc in range(len(tpcs)):
            data.extend(Transforms[tpc](Pts[tpc][event]))
        return data

files = [f for f in os.listdir(parsed_dir) if file_info(f)[2] == "spacepoint"][:10]
print(files)
T0s, Eff_sp, Pur_sp = [], [], []
for ti in np.linspace (-7, -5, 5):
    for tj in np.linspace(-7, -5, 5):
        Eff_sp_tmp, Pur_sp_tmp = [], []
        for sp_f in files:
            index = file_info(sp_f)[1]
            for event in range(5):
                with np.load(parsed_dir+'/'+sp_f) as f:
                    voxel_spacepoint = {tuple(pt): None for pt in f["arr_%d"%event]}
                joined = join_tpcs(index, event, (ti, tj))
                voxel_truth = {tuple(pt[:3]): pt[3]*1/3 for pt in joined}
                voxel_truth, voxel_spacepoint = util.filter_voxels_coord(((-100, 700), None, None), *util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True))
                #voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
                pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
                Pur_sp_tmp.append(pur_sp)
                Eff_sp_tmp.append(eff_sp)
        T0s.append((ti, tj))
        Eff_sp.append(np.mean(Eff_sp_tmp))
        Pur_sp.append(np.mean(Pur_sp_tmp))

                


In [ ]:
print(T0s)
print(Eff_sp)
print(Pur_sp)
fig, axes = plt.subplots(2)
axes[0].plot(Eff_sp, 'bo')
axes[1].plot(Pur_sp, 'bo')

In [ ]:
Eff_sp = np.array(Eff_sp)
Pur_sp = np.array(Pur_sp)
T0s=np.array(T0s)
args = np.argsort(Eff_sp)[:10:-1]
print(T0s[args])
print(Eff_sp[args])
print(Pur_sp[args])

## WIRECELL

In [ ]:
index = 1
event = 0
#index = 1
#event = 4
lims=None
#lims=((100, 400), (0,400), None)
#lims = ((600, 800), (300, 400), (0, 100))
yinf.plot_yinf_simple(plt.figure(), None, event, 0.35, 1, 0.5/3, lims, 
                      xy_dir=SIM_DIR+"reco_1GeV_BeamCosmic_xy/", yinf_file=ML_DIR+"singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-batch0-reco_BeamCosmic_%03d_yinf-TPC1.npy"%index)
products.plot_spacepoint(plt.figure(), None,event , (8, 4, 4), True, 20, 1, 0.5/3, lims, 
                    spacepoint_file=SIM_DIR+"wirecell_parsed/uniform/reco_1GeV_BeamCosmic_%03d_wirecell-TPC1.npz"%index, truth_file=SIM_DIR+"reco_1GeV_BeamCosmic_parsed/reco_BeamCosmic_%03d_depo-TPC1.npz"%index)

In [ ]:
index = 953
event = 10
#index = 1
#event = 4
lims=None
#lims=((100, 400), (0,400), None)
#lims = ((600, 800), (300, 400), (0, 100))
yinf.plot_yinf_simple(plt.figure(), None, event, 0.35, 1/3, 0.5/3, lims, 
                      xy_dir=SIM_DIR+"reco_1GeV_ElectronWire_xy-v3_3", yinf_file=ML_DIR+"singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-batch0-reco_singleElectron_%03d_yinf-TPC1.npy"%index)
products.plot_spacepoint(plt.figure(), None,event , (8, 4, 4), True, 10, 1/3, 0.5/3, lims, 
                    spacepoint_file=SIM_DIR+"wirecell_parsed/center/reco_1GeV_singleElectron_%03d_wirecell-TPC1.npz"%index, truth_file=SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_%03d_energy-TPC1.npz"%index)

Following cells are for tuning geom

In [ ]:
wirecell_dir = SIM_DIR+"wirecell_parsed/uniform-old"
parsed_dir=SIM_DIR+"reco_1GeV_BeamCosmic_parsed"
files = [f for f in os.listdir(wirecell_dir) if "BeamCosmic" in f]
print(files)
dXs, Eff_sp, Pur_sp = [], [], []
for dx in range(-2, 18, 2):
    Eff_sp_tmp, Pur_sp_tmp = [], []
    for sp_f in files:
        try:
            index = file_info(sp_f)[1]
        except Exception:
            continue
        true_file = parsed_dir+'/reco_BeamCosmic_%03d_depo-TPC1.npz'%index
        for event in range(5):
            with np.load(wirecell_dir+'/'+sp_f) as f:
                voxel_spacepoint = {(pt[0]+dx, pt[1], pt[2]): None for pt in f["arr_%d"%event]}
            with np.load(true_file) as f: 
                voxel_truth = {tuple(pt[:3]): pt[3]*1 for pt in f["arr_%d"%event]}
            #voxel_truth, voxel_spacepoint = util.filter_voxels_coord(((-100, 700), None, None), *util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True))
            #voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
            pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
            Pur_sp_tmp.append(pur_sp)
            Eff_sp_tmp.append(eff_sp)
    #print(Eff_sp_tmp)
    dXs.append(dx)
    Eff_sp.append(np.mean(Eff_sp_tmp))
    Pur_sp.append(np.mean(Pur_sp_tmp))


In [ ]:
wirecell_dir = SIM_DIR+"wirecell_parsed/uniform-old"
parsed_dir=SIM_DIR+"reco_1GeV_Electron_parsed-v3"
files = [f for f in os.listdir(wirecell_dir) if "Electron" in f][:10]
dXs, Eff_sp, Pur_sp = [], [], []
for dx in range(-2, 18, 2):
    Eff_sp_tmp, Pur_sp_tmp = [], []
    for sp_f in files:
        try:
            index = file_info(sp_f)[1]
        except Exception:
            continue
        true_file = parsed_dir+'/reco_singleElectron_%03d_energy-TPC1.npz'%index
        for event in range(30):
            with np.load(wirecell_dir+'/'+sp_f) as f:
                voxel_spacepoint = {(pt[0]+dx, pt[1], pt[2]): None for pt in f["arr_%d"%event]}
            with np.load(true_file) as f: 
                voxel_truth = {tuple(pt[:3]): pt[3]*1/3 for pt in f["arr_%d"%event]}
            #voxel_truth, voxel_spacepoint = util.filter_voxels_coord(((-100, 700), None, None), *util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True))
            #voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
            pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
            Pur_sp_tmp.append(pur_sp)
            Eff_sp_tmp.append(eff_sp)
    #print(Eff_sp_tmp)
    dXs.append(dx)
    Eff_sp.append(np.mean(Eff_sp_tmp))
    Pur_sp.append(np.mean(Pur_sp_tmp))


In [ ]:
print(dXs)
print(Eff_sp)
print(Pur_sp)
fig, axes = plt.subplots(2)
axes[0].plot(Eff_sp, 'bo')
axes[1].plot(Pur_sp, 'bo')

In [ ]:
with np.load("stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-BeamCosmic-tpc1_stats-thres0.17-comptrue-N500-all.npz",
             allow_pickle=True) as f:
    stats = f["event_stats"]
inf_thres, wirecell_thres = 18, 20
Eff_inf, Pur_inf = [], []
Eff_sp, Pur_sp = [], []
for evt in stats[:]:
    index, event = evt[:2]
    try:
        with np.load(SIM_DIR+"wirecell_parsed/uniform/reco_1GeV_BeamCosmic_%03d_wirecell-TPC1.npz"%index) as f:
            voxel_spacepoint = {tuple(pt[:3]): pt[3] for pt in f["arr_%d"%event]}
    except Exception:
        continue
    with np.load(SIM_DIR+"reco_1GeV_BeamCosmic_parsed/reco_BeamCosmic_%03d_depo-TPC1.npz"%index) as f:
        voxel_truth = {tuple(pt[:3]): pt[3]*1 for pt in f["arr_%d"%event]}
    voxel_truth, voxel_spacepoint = util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True)
    voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
    voxel_spacepoint = util.filter_voxel_val(voxel_spacepoint, wirecell_thres)
    pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
    Pur_sp.append(pur_sp)
    Eff_sp.append(eff_sp)
    Pur_inf.append(evt[2][inf_thres])
    Eff_inf.append(evt[3][inf_thres])

fig, axes = plt.subplots(2)
axes[0].scatter(Eff_inf, Eff_sp, s=2)
axes[1].scatter(Pur_inf, Pur_sp, s=2)
axes[0].plot([0,1], [0, 1], 'k:')
axes[1].plot([0,1], [0, 1], 'k:')
for ax in axes:
    ax.set_xlim((0,1))
    ax.set_ylim((0,1))
    ax.set_xlabel("Network (thres=0.36)")
    ax.set_ylabel("Spacepoint (thres=20)")
axes[0].set_title("Sensitivity")
axes[1].set_title("Purity")
fig.text(0.2, 0.8, "Network mean: %.3f"%np.mean(Eff_inf))
fig.text(0.2, 0.7, "Spacepoint mean: %.3f"%np.mean(Eff_sp))
fig.text(0.2, 0.3, "Network mean: %.3f"%np.mean(Pur_inf))
fig.text(0.2, 0.2, "Spacepoint mean: %.3f"%np.mean(Pur_sp))
fig.suptitle("Wirecell vs. Network \n file: epoch24-BeamCosmic-tpc1_stats-thres0.17-comptrue-N500-v2.npz")
plt.show()

In [ ]:
with np.load("stats/singleE-BeamCosmic-tpc1_ghost3D-164-lr0.01/inference/epoch24-Electron_stats-thres0.50-comptrue-N500-v2.npz",
             allow_pickle=True) as f:
    stats = f["event_stats"]
inf_thres, wirecell_thres = 18, 20
Eff_inf, Pur_inf = [], []
Eff_sp, Pur_sp = [], []
for evt in stats[:]:
    index, event = evt[:2]
    try:
        with np.load(SIM_DIR+"wirecell_parsed/uniform/reco_1GeV_singleElectron_%03d_wirecell-TPC1.npz"%index) as f:
            voxel_spacepoint = {tuple(pt[:3]): pt[3] for pt in f["arr_%d"%event]}
    except Exception:
        continue
    with np.load(SIM_DIR+"reco_1GeV_Electron_parsed-v3/reco_singleElectron_%03d_energy-TPC1.npz"%index) as f:
        voxel_truth = {tuple(pt[:3]): pt[3]*1/3 for pt in f["arr_%d"%event]}
    voxel_truth, voxel_spacepoint = util.downsample_voxels((8, 4, 4), voxel_truth, voxel_spacepoint, reduce=True)
    voxel_truth = util.filter_voxel_val(voxel_truth, 0.17)
    voxel_spacepoint = util.filter_voxel_val(voxel_spacepoint, wirecell_thres)
    pur_sp, eff_sp = util.SP_stats(voxel_truth, voxel_spacepoint, False, False)
    Pur_sp.append(pur_sp)
    Eff_sp.append(eff_sp)
    Pur_inf.append(evt[2][inf_thres])
    Eff_inf.append(evt[3][inf_thres])

fig, axes = plt.subplots(2)
axes[0].scatter(Eff_inf, Eff_sp, s=2)
axes[1].scatter(Pur_inf, Pur_sp, s=2)
axes[0].plot([0,1], [0, 1], 'k:')
axes[1].plot([0,1], [0, 1], 'k:')
for ax in axes:
    ax.set_xlim((0,1))
    ax.set_ylim((0,1))
    ax.set_xlabel("Network (thres=0.36)")
    ax.set_ylabel("Spacepoint (thres=20)")
axes[0].set_title("Sensitivity")
axes[1].set_title("Purity")
fig.text(0.2, 0.8, "Network mean: %.3f"%np.mean(Eff_inf))
fig.text(0.2, 0.7, "Spacepoint mean: %.3f"%np.mean(Eff_sp))
fig.text(0.2, 0.3, "Network mean: %.3f"%np.mean(Pur_inf))
fig.text(0.2, 0.2, "Spacepoint mean: %.3f"%np.mean(Pur_sp))
fig.suptitle("Wirecell vs. Network \n file: epoch24-Electron_stats-thres0.50-comptrue-N500-v2.npz")
plt.show()